In [ ]:
%pip install pymongo
%pip install yfinance
%pip install pandas


In [66]:
from pymongo import MongoClient, UpdateOne
from pymongo.errors import ServerSelectionTimeoutError
import yfinance as yf
import pandas as pd
import os
import datetime

MONGO_URI = "mongodb://127.0.0.1:27017/?directConnection=true"
client = MongoClient(MONGO_URI)
db = client["stocks_db"]
collection = db["daily_prices"]

In [67]:
try:
    print(client.admin.command({"ping": 1}))
except ServerSelectionTimeoutError as e:
    print("MongoDB not reachable:", e)
    raise

{'ok': 1.0}


In [68]:
# Ensure efficient upserts
collection.create_index([("ticker", 1), ("date", 1)], unique=True)

'ticker_1_date_1'

In [69]:
tickers_list = ["AAPL", "MSFT","AMZN"]  # You can modify this list to include more tickers
df = yf.download(tickers_list, start="2025-01-01", end="2025-02-01", interval="1d", progress=True)
df

/var/folders/4x/4_sg86ks701gcq77xn2dr5dc0000gn/T/ipykernel_78772/2913165074.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers_list, start="2025-01-01", end="2025-02-01", interval="1d", progress=True)
[*********************100%***********************]  3 of 3 completed


Price            Close                                High              \
Ticker            AAPL        AMZN        MSFT        AAPL        AMZN   
Date                                                                     
2025-01-02  243.263199  220.220001  416.976868  248.500565  225.149994   
2025-01-03  242.774368  224.190002  421.728607  243.592387  225.360001   
2025-01-06  244.410416  227.610001  426.211365  246.734810  228.839996   
2025-01-07  241.627136  222.110001  420.752350  244.959095  228.380005   
2025-01-08  242.115952  222.130005  422.933990  243.123531  223.520004   
2025-01-10  236.280045  218.940002  417.345459  239.582077  221.710007   
2025-01-13  233.835922  218.460007  415.592194  234.105277  219.399994   
2025-01-14  232.718613  217.759995  414.078033  235.551775  221.820007   
2025-01-15  237.297562  223.350006  424.677277  238.384950  223.570007   
2025-01-16  227.710693  220.660004  422.953857  237.437230  224.649994   
2025-01-17  229.426559  225.940002  427.386871  231.730998  226.509995   
2025-01-21  222.104218  230.710007  426.858856  223.879933  231.779999   
2025-01-22  223.291367  235.009995  444.491119  223.580662  235.440002   
2025-01-23  223.121780  235.419998  444.999115  226.483666  235.520004   
2025-01-24  222.243881  234.850006  442.359283  225.087029  236.399994   
2025-01-27  229.306854  235.419998  432.895660  231.591337  235.610001   
2025-01-28  237.686630  238.149994  445.487274  239.611994  241.770004   
2025-01-29  238.783981  237.070007  440.635895  239.282778  240.389999   
2025-01-30  237.018234  234.639999  413.400604  240.210530  237.949997   
2025-01-31  235.432083  237.679993  413.470367  246.595158  240.289993   

Price                          Low                                Open  \
Ticker            MSFT        AAPL        AMZN        MSFT        AAPL   
Date                                                                     
2025-01-02  424.438202  241.238085  218.190002  413.261173  248.330961   
2025-01-03  422.405996  241.307905  221.619995  417.933202  242.774368   
2025-01-06  432.656586  242.614744  224.839996  423.850447  243.722074   
2025-01-07  429.000637  240.769205  221.460007  419.188356  242.395272   
2025-01-08  425.334764  239.472335  220.199997  419.925568  241.337830   
2025-01-10  423.083378  232.439303  216.500000  413.430488  239.432429   
2025-01-13  416.897174  229.167192  216.470001  410.710966  232.968021   
2025-01-14  418.132423  231.910564  216.199997  409.146979  234.185076   
2025-01-15  426.510226  233.865838  220.750000  416.668060  234.075339   
2025-01-16  427.845056  227.481251  220.309998  422.764613  236.778830   
2025-01-17  432.816011  227.930169  223.080002  426.530180  231.561409   
2025-01-21  429.249658  218.852068  226.940002  423.969969  223.460945   
2025-01-22  445.556998  219.261080  231.190002  434.330172  219.261080   
2025-01-23  445.038970  221.765053  231.509995  439.809078  224.199183   
2025-01-24  444.939360  220.877183  232.929993  439.709467  224.239068   
2025-01-27  433.533224  223.440999  225.860001  421.878022  223.480912   
2025-01-28  446.662748  230.254561  233.979996  429.727856  230.294474   
2025-01-29  445.168487  233.446850  236.149994  438.713294  233.556586   
2025-01-30  421.240458  236.639159  232.220001  411.577626  238.095637   
2025-01-31  419.078810  232.878246  236.410004  413.320948  246.595158   

Price                                  Volume                      
Ticker            AMZN        MSFT       AAPL      AMZN      MSFT  
Date                                                               
2025-01-02  222.029999  423.900262   55740700  33956600  16896500  
2025-01-03  222.509995  419.467282   40244100  27515600  16662900  
2025-01-06  226.779999  426.360784   45045600  31849800  20573600  
2025-01-07  227.899994  427.356962   40856000  28084200  18139100  
2025-01-08  223.190002  421.838197   37628900  25033300  15054600  
2025-01-10  221.460007  423.003698   61710900  368

In [75]:
df_stacked = df.stack()
df_stacked.reset_index(inplace=True)
df_stacked

/var/folders/4x/4_sg86ks701gcq77xn2dr5dc0000gn/T/ipykernel_78772/1135859342.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_stacked = df.stack()


Price,Date,Ticker,Close,High,Low,Open,Volume
0,2025-01-02,AAPL,243.263199,248.500565,241.238085,248.330961,55740700
1,2025-01-02,AMZN,220.220001,225.149994,218.190002,222.029999,33956600
2,2025-01-02,MSFT,416.976868,424.438202,413.261173,423.900262,16896500
3,2025-01-03,AAPL,242.774368,243.592387,241.307905,242.774368,40244100
4,2025-01-03,AMZN,224.190002,225.360001,221.619995,222.509995,27515600
5,2025-01-03,MSFT,421.728607,422.405996,417.933202,419.467282,16662900
6,2025-01-06,AAPL,244.410416,246.734810,242.614744,243.722074,45045600
7,2025-01-06,AMZN,227.610001,228.839996,224.839996,226.779999,31849800
8,2025-01-06,MSFT,426.211365,432.656586,423.850447,426.360784,20573600
9,2025-01-07,AAPL,241.627136,244.959095,240.769205,242.395272,40856000


In [ ]:
# Download the data
tickers_list = ["AAPL", "MSFT"]  # You can modify this list to include more tickers
df = yf.download(tickers_list, start="2025-01-01", end="2025-02-01", interval="1d", progress=True)

if df.empty:
    print("No data returned from yfinance.")
else:
    # Flatten column names if they are MultiIndex (tuples)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [col[0].lower() for col in df.columns]
    else:
        df.columns = [col.lower() for col in df.columns]
    
    # Add ticker and reset index
    df['ticker'] = "AAPL"
    df.reset_index(inplace=True)
    
    # Make sure the date column is also lowercase
    df.columns = [col.lower() for col in df.columns]
    
    ops = []
    for record in df.to_dict("records"):
        doc = {
            "ticker": record["ticker"],
            "date": record["date"], # Now lowercase because we lowercased all columns
            "open": record["open"],
            "high": record["high"],
            "low": record["low"],
            "close": record["close"],
            "volume": record["volume"]
        }
        ops.append(
            UpdateOne(
                {"ticker": doc["ticker"], "date": doc["date"]},
                {"$set": doc},
                upsert=True
            )
        )

    res = collection.bulk_write(ops, ordered=False)
    print(f"Upserted: {res.upserted_count}, Modified: {res.modified_count}")

    # Verify by fetching a few records
    print("\nSample of inserted data:")
    for item in collection.find({"ticker": "AAPL"}).sort("date", 1).limit(50):
        print(item)

In [ ]:
result = collection.find_one(
    {
        "ticker": "AAPL",
        "date": datetime.datetime(2025, 1, 2)  # Example date to check
    }
)

print(result['volume'])

In [ ]:
delete_result = collection.delete_many({})
print(f"Deleted {delete_result.deleted_count} documents from the 'daily_prices' collection.")

# You can verify by counting the documents remaining (should be 0)
remaining_count = collection.count_documents({})
print(f"Current document count in collection: {remaining_count}")

In [ ]:
df